In [161]:
from gpcam.autonomous_experimenter import AutonomousExperimenterGP
import numpy as np
import plotly.graph_objects as go
import numpy as np

In [162]:
%matplotlib notebook
from pyScatt.packages import *
plt.rcParams.update({'figure.max_open_warning': 0})
plt.rcParams.update({ 'image.origin': 'lower'   })
plt.rcParams.update({ 'image.interpolation': 'none'   })
T = True
F = False

In [163]:
def plot(x,y,z,data = None):
    fig = go.Figure()
    fig.add_trace(go.Surface(x = x, y = y,z=z))
    if data is not None: 
        fig.add_trace(go.Scatter3d(x=data[:,0], y=data[:,1], z=data[:,2],
                                   mode='markers'))
    fig.update_layout(title='Posterior Mean', autosize=True,
                  width=800, height=800,
                  margin=dict(l=65, r=50, b=65, t=90))
    fig.show()



def kernel(x1,x2,hyperparameters,obj):
    hps = hyperparameters
    distance_matrix = np.zeros((len(x1),len(x2)))
    
    for i in range(len(x1[0])): distance_matrix += abs(np.subtract.outer(x1[:,i],x2[:,i])/hps[1+i])**2
    distance_matrix = np.sqrt(distance_matrix)
    noise = 0.0
    if len(x1) == len(x2): noise = np.identity(len(x1)) * hps[4]
    return   hps[0] *  obj.matern_kernel_diff1(distance_matrix,1) #+ noise

In [164]:
def convert_dict_to_data(D):
    return   np.array(list(D.values()))

def convert_data_dict(data):
    try:
        old_dict = h5todict(  'send_backup.h5'  ) 
        N = len( old_dict.keys() )
    except:
        N = 0
    D = {}
    i = 0 
    for d in data:
        D[ '%08d_'%(N+i) + d['id']] = d    
        i +=1
    return   D

def send_command_to_reactor(D):
    dicttoh5( D, 'send_backup.h5', mode='a' ) 
    dicttoh5( D, 'send.h5', mode='a' ) 

def get_analysis_results(   ):    
    return h5todict(  'received.h5'  ) 

# def extract_recipe(D):
#     D2 = {}
#     for k in D.keys():
#         D2[k] = D['position']
#     return D2
  

#def use_s3(data, instrument_dict = {}):
def use_s3(data):
    D = convert_data_dict(data)
    send_command_to_reactor(D)   
    print("successfully sent data")
    #np.save("sent",data)
    print("Sent and waiting for the results: ", D) 
    success = False
    while success is False:
         try:
            D = get_analysis_results(   )
            data = convert_dict_to_data(D) 
            success =  True
            os.remove( 'received.h5'  )  
            os.remove( 'send.h5'  )  
            
         except Exception as e:
             print("error in get() data")
             print(e)
             time.sleep( 3 )
    print("Successfully received data")
    #np.save("received",data)
    #print("data received: ", data)
    print("data received: ", D)
    #print("data received: ", list(D.keys()))
    return list(data)

def instrument(data, instrument_dict = {}):
    if "gp_obj" in instrument_dict:
        gp = instrument_dict["gp_obj"]
        print("In this iteration, the instrument has access to the gp object and can create its own data")
        print("So go ahead and fill in what you can, don't forget to set the key 'measured' to True")
        
        #print( data )
        data = use_s3(data)
        for entry in data:
            if entry["measured"] is False:
                print("Filling data values that could not be provided by the instrument")
                entry["value"] = gp.posterior_mean(
                    entry["position"])["f(x)"][0]
                entry["variance"] = gp.posterior_covariance(
                    entry["position"])["v(x)"][0] 
    else:
        print("This is the FIRST iteration, we'll be filling in sensible placeholders")        
        data = use_s3(data)
    return data




In [165]:
##set up your parameter space
parameters = np.array([[1.0,30.0], #Au
                       [0.0,35.0], #Cit Acid 
                       [0.0,35.0]]) #cit Na



##set up some hyperparameters, if you have no idea, set them to 1 and make the training bounds large
init_hyperparameters = np.array([1.,1.,1.,1.,1.])
hyperparameter_bounds =  np.array([[0.01,10],
                                   [1.,1000.],
                                   [1.,1000.],
                                   [1.,1000.],
                                   [0.0001,1.]])


##let's initialize the autonomous experimenter ...
my_ae = AutonomousExperimenterGP(parameters, init_hyperparameters,
      hyperparameter_bounds,instrument_func = instrument, 
                                 kernel_func= kernel,
       init_dataset_size= 2, communicate_full_dataset=True)


# #...train...
# my_ae.train()


This is the FIRST iteration, we'll be filling in sensible placeholders
successfully sent data
Sent and waiting for the results:  {'00000000_5e39e5f3-1328-4ad1-9055-e6be80a182d0': {'position': array([15.18080498, 21.31462009, 20.69497949]), 'value': None, 'variance': None, 'cost': None, 'id': '5e39e5f3-1328-4ad1-9055-e6be80a182d0', 'time stamp': 1658369903.4961045, 'date time': '20/07/2022_22:18:23', 'measured': False}, '00000001_1997218c-fac2-470e-8d71-dc75e4050ed1': {'position': array([12.56906101, 19.98264811,  5.86326901]), 'value': None, 'variance': None, 'cost': None, 'id': '1997218c-fac2-470e-8d71-dc75e4050ed1', 'time stamp': 1658369903.496974, 'date time': '20/07/2022_22:18:23', 'measured': False}}
error in get() data
Unable to open file (unable to open file: name = 'received.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
error in get() data
Unable to open file (unable to open file: name = 'received.h5', errno = 2, error message = 'No such f

In [166]:
my_ae.train()

In [167]:
my_ae.instrument_dict = {"gp_obj":my_ae.gp_optimizer}

In [168]:
my_ae.go(N = 4, retrain_globally_at=[30,40,50,60,70,80,100,120,150,200,300,500])

/opt/anaconda3/envs/pyScattering/lib/python3.7/site-packages/distributed/node.py:182: UserWarning:

Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43496 instead



This is the FIRST iteration, we'll be filling in sensible placeholders
successfully sent data
Sent and waiting for the results:  {'00000002_5e39e5f3-1328-4ad1-9055-e6be80a182d0': {'cost': {}, 'date time': '20/07/2022_22:18:23', 'id': '5e39e5f3-1328-4ad1-9055-e6be80a182d0', 'measured': array(True), 'position': array([15.18080498, 21.31462009, 20.69497949]), 'time stamp': 1658369903.4961045, 'value': 3.5577560461096613, 'variance': 1.8862014860851057}, '00000003_1997218c-fac2-470e-8d71-dc75e4050ed1': {'cost': {}, 'date time': '20/07/2022_22:18:23', 'id': '1997218c-fac2-470e-8d71-dc75e4050ed1', 'measured': array(True), 'position': array([12.56906101, 19.98264811,  5.86326901]), 'time stamp': 1658369903.496974, 'value': 60.26851372322607, 'variance': 7.763279830279601}, '00000004_e1602d99-6f89-4d2d-af51-21dc100954f9': {'position': array([10.3159429 , 14.69968535, 34.93648143]), 'value': None, 'variance': None, 'cost': None, 'id': 'e1602d99-6f89-4d2d-af51-21dc100954f9', 'time stamp': 165836

## Generate fake data using another pipeline

In [ ]:
# Dall = h5todict('send_backup.h5')
# ks = sorted(list(Dall.keys()))
# print( ks , len(ks ))

# D = h5todict('send.h5')
# ks = sorted(list(D.keys()))

# try:
#     Db = h5todict('received_backup.h5')
#     ksb = sorted(list(Db.keys()))
#     print( len(ksb ))
# except:
#     ksb = []

# print( ks )
# print( len(ks))

# for k in ks:
#     if k not in ksb:
#         D[k  ]['value'] = np.random.random() * 100
#         D[k]['variance'] =  np.sqrt( D[k  ]['value'] )
#         D[k  ]['measured'] =  True

# dicttoh5( D, 'received_backup.h5', mode='a' ) 
# dicttoh5( D, 'received.h5', mode='a' )     



# for k in ks:
#     if k not in ksb:
#         #D[k  ]['value'] = np.random.random() * 100
#         #D[k]['variance'] =  np.sqrt( D[k  ]['value'] )
#         D[k  ]['measured'] =  False

# dicttoh5( D, 'received_backup.h5', mode='a' ) 
# dicttoh5( D, 'received.h5', mode='a' )  

In [169]:
x_pred = np.zeros((10000,3))
x = np.linspace(0,40,100)
y = np.linspace(0,40,100)
x,y = np.meshgrid(x,y)
slice1 = 20.
counter = 0
for i in  range(100):
    for j in range(100):
        x_pred[counter] = np.array([x[i,j],y[i,j],slice1])
        counter += 1

In [170]:
print(my_ae.gp_optimizer.hyperparameters)

[1.00000000e+01 5.27928523e+02 1.77836232e+02 1.19324348e+00
 4.32422982e-01]


In [171]:
res = my_ae.gp_optimizer.posterior_mean(x_pred)
f = res["f(x)"]
f = f.reshape(100,100)

plot(x,y,f)